In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from haversine import haversine
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import scipy.stats as stats
import folium
import itertools
import statistics
import string

import warnings
warnings.filterwarnings("ignore")

In [8]:
addresses = ['서울시 도봉구 방학로6길 25', 
           '서울시 도봉구 덕릉로63길 19',
           '서울시 도봉구 해등로 133',
           '서울시 도봉구 방학로3길 16',
           '서울시 도봉구 도봉로133길 42',
           '서울시 도봉구 덕릉로59나길 20',
           '서울시 도봉구 해등로16가길 32',
           '서울시 도봉구 우이천로34길 38',
           '서울시 도봉구 노해로 41길 9',
           '서울시 도봉구 도봉로 969',
           '서울시 도봉구 방학1동 마들로 656']

# 주소에 해당하는 위도, 경도를 return해주는 함수
def geocoding(address):
    '''
    input : 주소
    output : 
    '''
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    crd = (float(geo.latitude), float(geo.longitude))

    return crd

def geo_df_coding(addresses):
    geo_list = []
    for address in addresses:
        crd = geocoding(address)
        geo_list.append(crd)
    geo_df = pd.DataFrame(geo_list)
    geo_df.columns = ['latitude', 'longitude']
    
    for i in range(geo_df.shape[0]):
        geo_df.loc[i,'name'] = string.ascii_lowercase[i]
        
    geo_df = geo_df[['name','latitude','longitude']]
    
    return geo_df

geo_df = geo_df_coding(addresses)

# Naver Map API

In [3]:
# *-- Geocoding 활용 코드 --*
import json
import urllib
from urllib.request import Request, urlopen

# 주소에 geocoding 적용하는 함수를 작성.
def get_location(loc) :
    # 내 고유값임 (공개안되도록)
    client_id = 'wi1vd6qb31'
    client_secret = 'NWOOthvHMkC11OEqF93vhqXPzsTellMMEq08gtf4'

    url = f"https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" \
    			+ urllib.parse.quote(loc)
    
    # 주소 변환
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urlopen(request)
    res = response.getcode()
    
    if (res == 200) : # 응답이 정상적으로 완료되면 200을 return
        response_body = response.read().decode('utf-8')
        response_body = json.loads(response_body)
        # print(response_body)
        
        # 주소가 존재할 경우 total count == 1이 반환됨.
        if response_body['meta']['totalCount'] == 1 : 
        	# 위도, 경도 좌표를 받아와서 return해 줌.
            lat = response_body['addresses'][0]['y']
            lon = response_body['addresses'][0]['x']
            return (lon, lat)
        else :
            print('location not exist')
        
    else :
        print('ERROR')
        
# *-- Directions 5 활용 코드 --*

# option : 탐색옵션 [최대 3개, traoptimal(기본 옵션) 
# trafast 실시간 빠른길
# tracomfort 실시간 편한길
# traoptimal 실시간 최적
# traavoidtoll 무료 우선
# traavoidcaronly 자동차 전용도로 회피 우선

option = 'traoptimal'

def get_optimal_route(start, goal, option=option ) :

    # 내 고유값임 (공개안되도록)
    client_id = 'wi1vd6qb31'
    client_secret = 'NWOOthvHMkC11OEqF93vhqXPzsTellMMEq08gtf4'

    # start=/goal=/(waypoint=)/(option=) 순으로 request parameter 지정
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving?start={start[0]},{start[1]}&goal={goal[0]},{goal[1]}&option={option}"
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urllib.request.urlopen(request)
    res = response.getcode()
    
    if (res == 200) :
        response_body = response.read().decode('utf-8')
        return json.loads(response_body)
            
    else :
        print('ERROR')

In [24]:
# Example
# *-- 목적지까지의 종합 정보 추출(총 거리, 총 소요시간, 요금 등) --*
# 총거리 : 전체 경로 거리(meters)
# 총소요시간 : 	전체 경로 소요 시간(milisecond(1/1000초))
message = {
    'message' : results['message'], # traoptimal은 option 에 따라 다르게 줄 것
    'option' : list(results['route'].keys())[0],
    'total_distance' : results['route']['traoptimal'][0]['summary']['distance'],
    'total_duration' : results['route']['traoptimal'][0]['summary']['duration'],
    'fares' : {'toll' : results['route']['traoptimal'][0]['summary']['tollFare'],
              'taxi' : results['route']['traoptimal'][0]['summary']['taxiFare'],
              'fuel' : results['route']['traoptimal'][0]['summary']['fuelPrice']}
}
print(message)

{'message': '길찾기를 성공하였습니다.', 'option': 'traoptimal', 'total_distance': 403827, 'total_duration': 19209108, 'fares': {'toll': 22100, 'taxi': 370010, 'fuel': 59173}}


In [25]:
# Example
# *-- 목적지까지의 guidence와 각각의 거리, 소요시간 정보 추출 --*
temp = [ (guide['instructions'], guide['distance'] , guide['duration'] / 1000 ) \
for guide in results['route']['traoptimal'][0]['guide'] ]

print(temp)

[("'창의문로' 방면으로 우회전", 12, 2.879), ('우회전', 1117, 161.99), ("세검정에서 '북악터널' 방면으로 오른쪽 방향", 1012, 172.187), ("국민대입구에서 '내부순환로' 방면으로 오른쪽 고가차도 진입", 4037, 603.958), ('도시고속도로 진입', 18, 2.212), ("하월곡분기점교차로에서 '북부간선도로(구리·월릉IC)' 방면으로 오른쪽 방향", 3636, 325.378), ("중랑IC에서 '남구리IC' 방면으로 오른쪽 고속도로 진입", 7209, 1197.352), ('중랑톨게이트 (통행료 1,400원)', 184, 10.95), ("남구리IC에서 '서울, 강변북로' 방면으로 오른쪽 고속도로 출구", 5116, 250.988), ("아천IC에서 '구리암사대교, 올림픽대교, 구리시청·경찰서' 방면으로 오른쪽 방향", 766, 87.298), ("아천IC에서 '구리암사대교, 올림픽대교' 방면으로 왼쪽 방향", 168, 56.631), ("암사IC에서 '종합운동장, 천호대교, 춘천' 방면으로 오른쪽 도시고속도로 진입", 1476, 436.486), ("암사IC에서 '강일IC' 방면으로 왼쪽 방향", 185, 48.372), ("강일IC에서 '수도권제1순환선, 대전, 판교' 방면으로 오른쪽 고속도로 진입", 2870, 336.058), ("하남분기점에서 '중부고속도로, 대전, 광주(경안)' 방면으로 왼쪽 도로 주행", 5392, 444.851), ('동서울톨게이트', 3680, 159.97), ('분기도로 진입', 1760, 79.803), ("산곡분기점에서 '대전, 서이천, 곤지암' 방면으로 오른쪽 방향", 2079, 94.271), ("호법분기점에서 '강릉, 인천' 방면으로 오른쪽 방향", 33114, 1329.34), ("호법분기점에서 '강릉' 방면으로 왼쪽 방향", 177, 8.228), ("여주분기점에서 '충주, 양평' 방면으로 오른쪽 방향", 15208, 597.252), ("여주분기점에서 '충주' 

In [6]:
addresses = ['서울시 도봉구 방학로6길 25', 
           '서울시 도봉구 덕릉로63길 19',
           '서울시 도봉구 해등로 133',
           '서울시 도봉구 방학로3길 16',
           '서울시 도봉구 도봉로133길 42',
           '서울시 도봉구 덕릉로59나길 20',
           '서울시 도봉구 해등로16가길 32',
           '서울시 도봉구 우이천로34길 38',
           '서울시 도봉구 노해로 41길 9',
           '서울시 도봉구 도봉로 969',
           '서울특별시 도봉구 마들로 656']

address_list = []
for address in addresses:
    address_list.append(get_location(address))
    
dist_mat = np.zeros((geo_df.shape[0], geo_df.shape[0]))
time_mat = np.zeros((geo_df.shape[0], geo_df.shape[0]))

option = 'traoptimal'
for i in range(geo_df.shape[0]):
    for j in range(geo_df.shape[0]):
        if (i==j):
            dist_mat[i][j] = 0
            time_mat[i][j] = 0
        else:
            results = get_optimal_route(start = address_list[i], goal = address_list[j], option=option)

            dist_mat[i][j] = results['route']['traoptimal'][0]['summary']["distance"]
            time_mat[i][j] = results['route']['traoptimal'][0]['summary']["duration"]



In [10]:
dist_mat

array([[   0., 3396., 1487.,  576.,  916., 3648., 2567., 2693., 2533.,
        4243., 1892.],
       [3383.,    0., 2495., 3006., 2334.,  441., 1772., 2739., 2519.,
        6431., 4055.],
       [1595., 1935.,    0., 1218.,  546., 2187.,  397., 2146., 1858.,
        4643., 2267.],
       [1144., 2898.,  989.,    0.,  678., 3150., 2240., 2698., 2410.,
        3879., 1528.],
       [1164., 2389.,  480.,  678.,    0., 2641., 1542., 2199., 1911.,
        4949., 2287.],
       [3525.,  142., 2637., 3148., 2476.,    0., 1914., 2881., 2661.,
        6573., 4197.],
       [1767., 1827., 1550., 1390.,  718., 2079.,    0., 2038., 1750.,
        4815., 2439.],
       [4092., 2436., 2096., 2819., 2182., 2503., 2031.,    0.,  288.,
        6279., 3903.],
       [2577., 2148., 1808., 2566., 1894., 2283., 1743.,  288.,    0.,
        5991., 3615.],
       [3625., 6090., 4181., 3557., 4159., 6342., 5432., 5890., 5602.,
           0., 2938.],
       [2069., 4520., 2735., 1777., 2453., 4772., 3395., 418

In [11]:
time_mat

array([[      0.,  675082.,  473962.,  158295.,  221647.,  735180.,
         840963.,  711327.,  672623.,  794367.,  752267.],
       [ 785601.,       0.,  843094.,  690345.,  708479.,  102742.,
         469083.,  743088.,  686805., 1309237., 1070461.],
       [ 494318.,  406674.,       0.,  438102.,  174205.,  457052.,
         100052.,  533580.,  466169.,  909655.,  763393.],
       [ 353382.,  730940.,  336981.,       0.,  225525.,  644210.,
         878028.,  666590.,  599179.,  689380.,  606122.],
       [ 368384.,  590484.,  196525.,  183169.,       0.,  640862.,
         509501.,  515926.,  448515., 1056600.,  782435.],
       [ 807070.,   35906.,  879001.,  715477.,  744386.,       0.,
         504990.,  778995.,  722712., 1341608., 1060899.],
       [ 607080.,  379861.,  487023.,  510313.,  286967.,  430239.,
              0.,  506767.,  439356.,  978916.,  835604.],
       [ 777599.,  746469.,  816527.,  899133.,  808431.,  851345.,
         789633.,       0.,  137692., 14204